In [2]:
# 필요한 패키지를 불러옵니다.

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
%matplotlib inline

In [3]:
# 텐서플로우 데이터셋의 cifar100 데이터 셋을 불러옵니다.
# 사전에 데이터 로드시 super class 라벨을 가져옵니다.

(train_images, train_labels), (valid_images, valid_labels) = tf.keras.datasets.cifar100.load_data(
    label_mode='coarse'
)

169001437/169001437 [==============================] - 4s 0us/step


In [ ]:
# 전이학습을 위한 전처리 함수를 정의합니다.

def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_images = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_images

In [ ]:
# 이미지를 전처리하여 할당합니다.

train_X = preprocess_image_input(train_images)
valid_X = preprocess_image_input(valid_images)

In [ ]:
# 미리 학습된 모델을 로드합니다.
feature_extractor = tf.keras.applications.resnet.ResNet50(
  input_shape=(224, 224, 3),
  include_top = False,
  weights='imagenet'
)

In [ ]:
# 특징추출기를 동결합니다.
feature_extractor.trainable = False

In [ ]:
# 함수형 케라스 API를 호출합니다.

inputs = tf.keras.Input(shape=(32, 32, 3))
x = tf.keras.layers.UpSampling2D(size=(7, 7))(inputs)
x = feature_extractor(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
outputs = tf.keras.layers.Dense(20, activation='softmax', name="classification")(x)

In [ ]:
model = tf.keras.Model(inputs = inputs, outputs= outputs)

In [ ]:
# optimizer로 확률적 경사하강법, 손실함수는 SparseCategoricalCrossentropy으로 컴파일합니다.

model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
# 사전 학습된 데이터의 input은 224 * 224 이므로 32 * 32 이미지를 업 샘플링하는 레이어를 추가합니다.
# GAP - FC 1024 - FC512 층과 레이어의 끝에 FC 20(coarse label)로 구성된 모델입니다.

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dense_1 (Dense)             (None, 512)               524800

In [ ]:
# 정확도 78%, 손실 0.68정도로 관측됩니다.
# 동결하였을 때, 학습속도는 빠르나 정확도가 아쉽습니다.

history = model.fit(train_X, train_labels, epochs=10,
                    validation_data=(valid_X, valid_labels), batch_size=64, shuffle=True)

Epoch 1/10
782/782 [==============================] - 35s 37ms/step - loss: 1.4823 - accuracy: 0.5336 - val_loss: 1.5903 - val_accuracy: 0.4962
Epoch 2/10
782/782 [==============================] - 27s 35ms/step - loss: 1.1020 - accuracy: 0.6461 - val_loss: 1.1720 - val_accuracy: 0.6262
Epoch 3/10
782/782 [==============================] - 27s 35ms/step - loss: 0.9852 - accuracy: 0.6805 - val_loss: 1.0537 - val_accuracy: 0.6562
Epoch 4/10
782/782 [==============================] - 27s 35ms/step - loss: 0.9095 - accuracy: 0.7064 - val_loss: 1.0284 - val_accuracy: 0.6717
Epoch 5/10
782/782 [==============================] - 27s 35ms/step - loss: 0.8502 - accuracy: 0.7257 - val_loss: 1.0107 - val_accuracy: 0.6748
Epoch 6/10
782/782 [==============================] - 28s 36ms/step - loss: 0.8028 - accuracy: 0.7418 - val_loss: 1.1257 - val_accuracy: 0.6431
Epoch 7/10
782/782 [==============================] - 28s 35ms/step - loss: 0.7583 - accuracy: 0.7565 - val_loss: 1.0034 - val_accuracy:

In [ ]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
# 동일한 형태에서 전체 레이어를 학습할 수 있도록 동결 부분을 제외하였습니다.
# 학습 시간은 동결할 때 보다 증가하였으나, 학습성능이 월등히 높은 것을 확인할 수 있습니다.
# 정확도는 99.96% 손실은 0.003 수준으로 매우 정확도가 높은 모델이라고 볼 수 있습니다. 

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
%matplotlib inline

(train_images, train_labels), (valid_images, valid_labels) = tf.keras.datasets.cifar100.load_data(
    label_mode='coarse'
)

def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_images = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_images

train_X = preprocess_image_input(train_images)
valid_X = preprocess_image_input(valid_images)

feature_extractor = tf.keras.applications.resnet.ResNet50(
  input_shape=(224, 224, 3),
  include_top = False,
  weights='imagenet'
)

# feature_extractor.trainable = False

inputs = tf.keras.Input(shape=(32, 32, 3))
x = tf.keras.layers.UpSampling2D(size=(7, 7))(inputs)
x = feature_extractor(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
outputs = tf.keras.layers.Dense(20, activation='softmax', name="classification")(x)

model = tf.keras.Model(inputs = inputs, outputs= outputs)

model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

history = model.fit(train_X, train_labels, epochs=10,
                    validation_data=(valid_X, valid_labels), batch_size=64, shuffle=True)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d_1 (UpSampling  (None, 224, 224, 3)      0         
 2D)                                                             
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_3 (Dense)             (None, 512)               5248